In [1]:
#import nbpresent
#nbpresent.__version__


In [2]:
import time

import bokeh.plotting
bokeh.plotting.output_notebook()
now=time.gmtime()
print("This notebook was last updated: "+time.asctime(now)+" UTC")

Loading BokehJS ...

This notebook was last updated: Mon Oct 17 22:51:47 2016 UTC


Let's compare catalogs

In [3]:
import astropy.io.votable as votab
import astropy.io.ascii 
import astropy.units as u

import numpy as np

#download latest target lists:

latest_RV_url = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/PlanetPopulation/RVplanets_ipac_2016-05-15.votable"
exoarch_confirmed_url = "http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets&format=votable&select=*"
exoplanets_eu_url = "http://www.exoplanet.eu/catalog/votable/" 
EXOSIM_stars = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/StarCatalog/mission_exocat.votable"
oec_latest_tab = "https://raw.githubusercontent.com/douglase/oec_tables/master/comma_separated/open_exoplanet_catalogue.txt"
#right now some BD objects in the main catalog have improperly formatted names, this is fixed in the branch above.

#my fork adds it and a pull request has been submitted: https://github.com/OpenExoplanetCatalogue/oec_tables/pull/3



#(right now this misses a couple of planets that are on multiple OEC lists and are confirmed and where there confirmed status is not the first entry. 
#FIX! 

In [4]:
#parse
exocat = votab.parse(EXOSIM_stars,datatype_mapping={"char": "unicodeChar"}).get_first_table().to_table()
#knownRV = votab.parse("../EXOSIMS/EXOSIMS/PlanetPopulation/RVplanets_ipac_032216.votable").get_first_table().to_table()
knownRV = votab.parse(latest_RV_url,datatype_mapping={"char": "unicodeChar"}).get_first_table().to_table()
#exoplanets_dot_eu = votab.parse(exoplanets_eu).get_first_table().to_table()
#create 

exoarch_confirmed_file=astropy.utils.data.download_file(exoarch_confirmed_url, 
                                                        cache=False,
                                                        show_progress=True,
                                                        timeout=10) #regular call was timing out
exoarchConfirmed = votab.parse(exoarch_confirmed_file,datatype_mapping={"char": "unicodeChar"}).get_first_table().to_table()
last_download_time=time.asctime(now)

In [5]:
if exoarchConfirmed["pl_orbper"].unit == "days":
    print("days not a recognized unit, replacing with day")
    exoarchConfirmed["pl_orbper"].unit=u.day #fix units
if knownRV["pl_orbper"].unit == "days":
    print("days not a recognized unit, replacing with day")
    knownRV["pl_orbper"].unit=u.day #fix units


days not a recognized unit, replacing with day
days not a recognized unit, replacing with day


In [6]:
'''exoplanets_eu_file = astropy.utils.data.download_file(exoplanets_eu_url, 
                                                        cache=False,
                                                        show_progress=True,
                                                        timeout=60) 
exoplanets_dot_eu = votab.parse(exoplanets_eu_file)#.get_first_table().to_table() #returns E05: Invalid boolean value 'Yes'
'''

"exoplanets_eu_file = astropy.utils.data.download_file(exoplanets_eu_url, \n                                                        cache=False,\n                                                        show_progress=True,\n                                                        timeout=60) \nexoplanets_dot_eu = votab.parse(exoplanets_eu_file)#.get_first_table().to_table() #returns E05: Invalid boolean value 'Yes'\n"

In [7]:
#Define open exoplanet catalog column names:
oec_cols=["name",#  1: Primary identifier of planet
          "binary",#  2: Binary flag [0=no known stellar binary companion; 1=P-type binary (circumbinary); 2=S-type binary; 3=orphan planet (no star)]
          "pl_massj",##  3: Planetary mass [Jupiter masses]
          "pl_radiusj",##  4: Radius [Jupiter radii]
          "pl_orbper",#  5: Period [days]
          "pl_orbsmax",#  6: Semi-major axis [Astronomical Units]
          "pl_orbeccen",#  7: Eccentricity
          "pl_orblper",#  8: Periastron [degree]
          "longitude", ##  9: Longitude [degree] https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/issues/292
          "ascendingnode",#"10: Ascending node [degree]",
          "pl_orbincl",#"# 11: Inclination [degree]
          "pl_eqt",# 12: Surface or equilibrium temperature [K]
          "pl_age",# 13: Age [Gyr]
          "pl_discmethod",#,14: Discovery method
          "pl_disc",# 15:" Discovery year [yyyy]
          "rowupdate",# 16: Last updated [yy/mm/dd]
          "ra_str",# 17: Right ascension [hh mm ss]
          "dec_str",# 18: Declination [+/-dd mm ss]
          "st_dist",# 19: Distance from Sun [parsec]
          "st_mass",# 20: Host star mass [Solar masses]
          "st_rad",# 21: Host star radius [Solar radii]
          "st_metfe",# 22: Host star metallicity [log relative to solar]
          "st_teff",# 23: Host star temperature [K],"24: Host star age [Gyr]"]
          "st_age",# 24: Host star age [Gyr]
          "list",#detection status list
]
          
oec=astropy.io.ascii.read(oec_latest_tab,data_start=31,names=oec_cols,delimiter=",",guess=False)
oec_all = oec.copy()
oec=oec[oec["list"]=="Confirmed planets"] #throw away planets that aren't marked confirmed
oec
oec["pl_orbper"].unit=u.day
oec["st_dist"].unit=u.parsec


In [8]:
def scatter_with_labels(p,x,y,names,**kwargs):
    #data source for mouseover
    if isinstance(names[0],bytes):
        names= [j.decode('utf-8') for j in names]
    source = bokeh.models.ColumnDataSource(data=dict(
        x=x,
        y=y,
        name=names, #convert bytes to strings
    ))

    p.scatter(x,y,line_color=None,source=source,**kwargs)
tooltips=[("index", "$index"),
          ("(x,y)", "(@x, @y)"),
          ("name", "@name")]

In [9]:

TOOLS="hover,crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"

p = bokeh.plotting.figure(tools=TOOLS, x_range=( 10000,0))

p.scatter(exocat['st_teff'], (exocat['st_mass']), radius=50,
          fill_color='gray', fill_alpha=0.5,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_mass"]), radius=100,
fill_color="orange", fill_alpha=0.2,legend="Known RV",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Mass [M_sun]"
bokeh.plotting.show(p)


In [10]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1e-3,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'],exocat['st_lbol'],
                    exocat["hip_name"],
                    color="blue",legend="exocat",size=10,fill_alpha=0.1)

scatter_with_labels(p,knownRV["st_teff"],10**(knownRV["st_lum"]), 
                    knownRV["pl_name"],
                    color="orange",legend="Known RV",size=10,fill_alpha=.3)

p.title.text = "H-R Diagram"
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Luminousity [L_sun]"
bokeh.plotting.show(p)


In [11]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))


p.scatter(exocat['st_teff'], (exocat['st_dist']), 
                              exocat["hip_name"],
          size=5,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_dist"]),
                              knownRV["pl_name"], size=10,
          fill_color="blue", fill_alpha=0.2,legend="Known RV",
          line_color=None)

p.scatter(exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), size=10,
fill_color="green", fill_alpha=0.1,legend="IPAC Exoplanet Archive",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)




In [12]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'], (exocat['st_dist']),
                    exocat["hip_name"],
                    radius=50,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",)
scatter_with_labels(p,knownRV['st_teff'], (knownRV["st_dist"]), knownRV["pl_name"],
                    radius=100,
                    fill_color="blue", fill_alpha=0.2,legend="Known RV",)
scatter_with_labels(p,exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), exoarchConfirmed["pl_hostname"],
                    radius=100,
                    fill_color="green", fill_alpha=0.2,legend="IPAC Exoplanet Archive",)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)



In [13]:
hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35),x_range=(0, 25),  )

scatter_with_labels(p,exoarchConfirmed["st_optmag"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="green",legend="IPAC Exoplanet Archive",
                    size=12,fill_alpha=0.34)

scatter_with_labels(p,knownRV["st_optmag"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    size=12,fill_alpha=0.34,
                    color="blue",legend="Known RV")

#bokeh.plotting.show(p)
p.xaxis.axis_label = "Optical Mag"
p.yaxis.axis_label = "distance [pc]"

#what is optical mag? Johnson v_mag?




In [14]:
hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35))#x_range=(0, 10000),  )

scatter_with_labels(p,exoarchConfirmed["pl_orbper"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="gray",legend="IPAC Exoplanet Archive",size=6,fill_alpha=0.6)

scatter_with_labels(p,knownRV["pl_orbper"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    color="blue",legend="Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p,oec["pl_orbper"],oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)
p.xaxis.axis_label = "orbital period [d]"
p.yaxis.axis_label = "distance [pc]"



In [15]:

def sep_as(dist,period):
    '''
    calculate the separation in arcseconds
    $p[years]^2=a[AU]^3$

    s["]=a[AU]/d[pc]

    $s["]=(p)^{2/3}/d$
    '''
    return (period.to(u.year).value**(2/3)/dist.to(u.parsec).value)
hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p2 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",  y_range=(-5, 45),x_range=(1e-2, 40),  )
#https://github.com/bokeh/bokeh/issues/3531

scatter_with_labels(p2,sep_as(exoarchConfirmed["st_dist"],exoarchConfirmed["pl_orbper"]),
                    exoarchConfirmed["st_dist"],
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    color="gray",legend="IPAC Exoplanet Archive",size=10,fill_alpha=0.6)

scatter_with_labels(p2,sep_as(knownRV["st_dist"],knownRV["pl_orbper"]),knownRV["st_dist"], 
                    knownRV["pl_hostname"] +knownRV["pl_letter"],
                    color="blue",legend="EXOSIMS Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p2,sep_as(oec["st_dist"],oec["pl_orbper"]),oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)
p2.xaxis.axis_label = "separation [arcseconds]"
p2.yaxis.axis_label = "distance [pc]"


p2.title.text = "Separation (from period, assuming circular orbits)"


bokeh.plotting.show(p2)

In [16]:
import pandas as pd
curated = pd.read_excel("../../projects/wfirst/WFIRST_prime_RV_targets_082016.xlsx",
                        header=0,skiprows=1, encoding="utf-8-sig",
                        index_col=None)
#curated["DIST"]

In [17]:
bokeh.io.curdoc().clear()#https://github.com/bokeh/bokeh/issues/3943

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p1 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",  y_range=(-5, 75),x_range=(1e-1, 40),  )

scatter_with_labels(p1,exoarchConfirmed["pl_msinij"],exoarchConfirmed["st_dist"],
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    size=12,fill_alpha=0.6,
                    color="gray",legend="IPAC Exoplanet Archive")

scatter_with_labels(p1,knownRV["pl_msinij"],knownRV["st_dist"], 
                     knownRV["pl_hostname"] +knownRV["pl_letter"],
                    size=12,fill_alpha=0.34,
                    color="blue",legend="EXOSIMS Known RV")

scatter_with_labels(p1,curated["MSINI"],curated["DIST"], 
                     curated["NAME"],
                    size=20,fill_alpha=0.34,
                    color="red",legend="curated",marker='square')

scatter_with_labels(p1,oec["pl_massj"],oec["st_dist"], 
                    np.array(oec["name"],dtype=object)+ np.array(oec["pl_discmethod"],dtype=object),
                    color="orange",legend="Open Exoplanet Catalog Confirmed Planets",size=10,fill_alpha=.35)
p1.xaxis.axis_label = "orbital period [d]"
scatter_with_labels(p1,oec["pl_massj"],oec["st_dist"],oec["name"],
                    size=10,color='orange',fill_alpha=.3)
p1.xaxis.axis_label = "msini [Jupiter]"
p1.yaxis.axis_label = "distance [pc]"
p1.title.text = "Catalogs last queried: "+last_download_time
bokeh.plotting.show(p1)

In [18]:
bokeh.io.curdoc().clear()#https://github.com/bokeh/bokeh/issues/3943


hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p3 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",y_axis_type="log",  
                           y_range=(0.3e-1, 40),x_range=(1e-2, 40),  )
#https://github.com/bokeh/bokeh/issues/3531

scatter_with_labels(p3,exoarchConfirmed["pl_msinij"],
                    sep_as(exoarchConfirmed["st_dist"],exoarchConfirmed["pl_orbper"]),
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    color="gray",legend="IPAC Exoplanet Archive (msini)",size=10,fill_alpha=0.6)
scatter_with_labels(p3,exoarchConfirmed["pl_bmassj"],
                    sep_as(exoarchConfirmed["st_dist"],exoarchConfirmed["pl_orbper"]),
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"]+ exoarchConfirmed["pl_discmethod"],        
                    color="gray",legend="IPAC Exoplanet Archive (Best Mass)",
                    size=10,fill_alpha=0.6,marker="diamond")

scatter_with_labels(p3,                    knownRV["pl_msinij"],
                    sep_as(knownRV["st_dist"],knownRV["pl_orbper"]),
                    knownRV["pl_hostname"] +knownRV["pl_letter"],
                    color="blue",legend="EXOSIMS Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p3,oec["pl_massj"], 
                    sep_as(oec["st_dist"],oec["pl_orbper"]),
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)

scatter_with_labels(p3,curated["MSINI"],
                    curated["Separation"], 
                     curated["NAME"],
                    size=20,fill_alpha=0.34,
                    color="red",legend="curated",marker='square')


p3.yaxis.axis_label = "separation [arcseconds]"
p3.xaxis.axis_label = "mass [Jupiter]"


p3.title.text = "Separation  (from period, assuming circular orbits) vs mass"


bokeh.plotting.show(p3)

In [19]:
grid = bokeh.layouts.gridplot([p2,p1,p3], ncols=2, plot_width=450, plot_height=600)
bokeh.plotting.output_file("pc_msini_and_sep.html", title=last_download_time)
#bokeh.plotting.show(p)
bokeh.plotting.show(grid)

KeyboardInterrupt: 

In [24]:
exoc_pd = exocat.to_pandas()
exoarc_pd = exoarchConfirmed.to_pandas()
knownRV_pd=knownRV.to_pandas()

oec_pd=oec.to_pandas()
oec_all_pd=oec_all.to_pandas()


In [25]:
print(exoc_pd.columns[exoc_pd.columns.str.contains("name")])

Index(['hip_name', 'hd_name', 'gj_name', 'tm_name'], dtype='object')


In [26]:
np.array(oec["name"],dtype=object)+np.array(oec["name"],dtype=object)

array(['6 Lyn b6 Lyn b', '61 Vir b61 Vir b', '61 Vir c61 Vir c', ...,
       'BD-08 2823 bBD-08 2823 b', 'BD-08 2823 cBD-08 2823 c',
       'BD-11 4672 bBD-11 4672 b'], dtype=object)

In [99]:
from astroquery.vizier import Vizier

def get_hip(target_name):
    viz=Vizier.query_object(target_name, catalog="I/311/hip2") 
    if len(viz.values()) ==0:
        return target_name
    return "HIP  "+str(viz['I/311/hip2']["HIP"][0])
get_hip("eps eri")

INFO:requests.packages.urllib3.connectionpool:Resetting dropped connection: vizier.u-strasbg.fr


'HIP  16537'

In [100]:
import cat_proc
from importlib import reload
reload(cat_proc)
#test that the Simbad query works (need internet...):
assert cat_proc.get_names("Epsilon Eridani",catalog="HIP") == 'HIP  16537'
assert get_hip("epsilon eridani") == 'HIP  16537'



INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): simbad.u-strasbg.fr


In [101]:
curated["hip_name"]=curated["NAME"]
for i in curated["hip_name"].index:
    name=curated.loc[i,"NAME"]
    curated.loc[i,"hip_name"]=(get_hip(name[:-2])+name[-2:]).strip("\n").strip()
        

In [73]:
oec_nearby_pd=oec_all_pd[(oec_all_pd["st_dist"]<50) & (oec_all_pd["pl_orbsmax"]>1)]
oec_nearby_pd["hip_name"]=oec_nearby_pd["name"]

/home/edouglas/miniconda2/envs/wfirst/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [102]:
for i in oec_nearby_pd["hip_name"].index:
    name = oec_nearby_pd.loc[i,"name"]

    oec_nearby_pd.loc[i,"hip_name"]=(get_hip(name[:-2])+name[-2:]).strip("\n").strip()
        

/home/edouglas/miniconda2/envs/wfirst/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/edouglas/miniconda2/envs/wfirst/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/edouglas/miniconda2/envs/wfirst/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

'BD-11 4672'

In [105]:
oec_nearby_pd_HP=oec_nearby_pd.set_index("hip_name",inplace=False)
curated_HP=curated.set_index("hip_name",inplace=False)

In [179]:
result = pd.merge(curated_HP, oec_nearby_pd_HP,how='inner',left_index=True,right_index=True)#,join='outer',axis=0)
result["%Mj"]=np.abs ((result["MSINI"]-result["pl_massj"])/result["MSINI"]*100)
result["%AU"]=np.abs((result["A"]-result["pl_orbsmax"])/result["A"]*100)

updates=result.copy()
result[["NAME","MSINI","pl_massj","%Mj","A","pl_orbsmax","%AU"]].sort_values("%Mj")




In [189]:
updates["ADSbibcode"]=" "
updates["Notes"]=" "

In [192]:
#update parameters
#HD 217107 c:
updates.loc['HIP  113421 c',"ADSbibcode"] = "2015ApJ...800...22F" 
updates.loc['HIP  113421 c',"A"] = 6.074 #HD 217107 c
updates.loc['HIP  113421 c',"MSINI"] = 4.513 #HD 217107 c

#eps eri:
updates.loc['HIP  16537 b',"ADSbibcode"] = "2012ApJS..200...15A" 
updates.loc['HIP  16537 b',"A"] = 3.51 #HD 217107 c
updates.loc['HIP  16537 b',"MSINI"] = 0.645 #HD 217107 c

#gamma ceph b:
# oec, exoplanets.eu and exoplanet archive all cite http://adsabs.harvard.edu/abs/2011AIPC.1331...88E
# the exoplanets.org value is from http://adsabs.harvard.edu/abs/2003ApJ...599.1383H
updates.loc['HIP  116727 b',"ADSbibcode"] = "2011AIPC.1331...88E" 
updates.loc['HIP  116727 b',"A"] = 2.05 #HD 217107 c
updates.loc['HIP  116727 b',"MSINI"] = 1.85 #HD 217107 c
updates["%AU"]=np.abs((updates["A"]-updates["pl_orbsmax"])/updates["A"]*100)




updates[["NAME","MSINI","pl_massj","%Mj","A","pl_orbsmax","%AU","ADSbibcode"]].sort_values("%Mj")

,NAME,MSINI,pl_massj,%Mj,A,pl_orbsmax,%AU,ADSbibcode
hip_name,,,,,,,,
HIP 88048 c,nu Oph c,27.000000,27.000000,0.000000,6.10000,6.100,0.000000,
HIP 522 c,HD 142 c,5.300000,5.300000,0.000000,6.80000,6.800,0.000000,
HIP 10626 b,HD 13931 b,1.881230,1.880000,0.065383,5.14931,5.150,0.013400,
HIP 90485 c,HD 169830 c,4.064070,4.040000,0.592263,3.60184,3.600,0.051085,
HIP 106440 b,GJ 832 b,0.684002,0.688908,0.717249,3.56153,3.600,1.080154,
HIP 95467 d,HD 181433 d,0.535482,0.540000,0.843726,3.02208,3.000,0.730623,
HIP 49699 b,HD 87883 b,1.755710,1.780000,1.383486,3.57648,3.600,0.657630,
HIP 7978 b,HD 10647 b,0.925037,0.940000,1.617557,2.02175,2.015,0.333869,
HIP 74500 c,HD 134987 c,0.804926,0.820000,1.872719,5.82513,5.800,0.431407,
